<a href="https://colab.research.google.com/github/eljimenezj/-Data-modeling-techniques/blob/master/Linear_Classifiers_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Clasificadores lineales en Python

En este caso vamos a estudiar unas aplicaciones de clasificación utilizando  modelos machine learning. Este libro de jupyter está basado en clases tomadas de datacamp, en la cual también utilizamos una base de datos de vinos que utilizan en varias de sus clases.

Este curso hace parte del track Machine Learning Scientist with Python

In [0]:
import warnings
warnings.filterwarnings("ignore")

Los primeros clasificadores que vamos a estudiar son regresion logistica y maquinas de soporte vectorial

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


digits = datasets.load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target)

# Apply logistic regression and print scores
lr = LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

1.0
0.9688888888888889


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [6]:

# Apply SVM and print scores
svm = SVC()
svm.fit(X_train, y_train)
print(svm.score(X_train, y_train))
print(svm.score(X_test, y_test))

0.9955456570155902
0.9888888888888889


Ahora haremos uso de probabilidades: El codigo necesita cargar los datos previamente para que funcione.

Fuente de datos: http://ai.stanford.edu/~amaas/data/sentiment/

Dateset: Este es un conjunto de datos para la clasificación de sentimientos binarios que contiene sustancialmente más datos que los conjuntos de datos de referencia anteriores. Proporcionamos un conjunto de 25,000 críticas de películas altamente polares para capacitación y 25,000 para pruebas. También hay datos adicionales sin etiquetar para su uso. Se proporcionan formatos de texto sin procesar y bolsa de palabras ya procesada. Consulte el archivo README contenido en la versión para obtener más detalles.

In [0]:
# Instantiate logistic regression and train
lr = LogisticRegression()
lr.fit(X,y)

# Predict sentiment for a glowing review
review1 = "LOVED IT! This movie was amazing. Top 10 this year."
review1_features = get_features(review1)
print("Review:", review1)
print("Probability of positive review:", lr.predict_proba(review1_features)[0,1])

# Predict sentiment for a poor review
review2 = "Total junk! I'll never watch a film by that director again, no matter how good the reviews."
review2_features = get_features(review2)
print("Review:", review2)
print("Probability of positive review:", lr.predict_proba(review2_features)[0,1])

Ahora realizaremos una visualizacion de los límites de decisión en los datos para saber si es un problema linealmente separable o no.

In [0]:
from sklearn.datasets import load_wine
wine = load_wine()

X = wine.data
y = wine.target


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier

# Define the classifiers
classifiers = [LogisticRegression(),LinearSVC(), SVC(), KNeighborsClassifier()]

# Fit the classifiers
for c in classifiers:
    c.fit(X,y)

# Grafica de funcion similiar a https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html , organizar.